In [49]:
#Here we import packages that we will need for this notebook. You can find out about these packages in the Help menu.

# although math is "built in" it needs to be imported so it's functions can be used.
import math

from scipy import constants, interpolate

#see numpy cheat sheet https://www.dataquest.io/blog/images/cheat-sheets/numpy-cheat-sheet.pdf
#The numpy import is needed because it is renamed here as np.
import numpy as np

import pandas as pd

import matplotlib.pyplot as plt

# sys and os give us access to operating system directory paths and to sys paths.
import sys, os

# If you place your GitHub directory in your documents folder and 
# clone both the design challenge notebook and the AguaClara_design repo, then this code should all work.
# If you have your GitHub directory at a different location on your computer, 
# then you will need to adjust the directory path below.
# add the path to your GitHub directory so that python can find files in other contained folders.
myGitHubdir=os.path.expanduser('~\\Documents\\GitHub')
if myGitHubdir not in sys.path:
    sys.path.append(myGitHubdir)

# add imports for AguaClara code that will be needed
# physchem has functions related to hydraulics, fractal flocs, flocculation, sedimentation, etc.
from AguaClara_design import physchem as pc

# pipedatabase has functions related to pipe diameters
from AguaClara_design import pipedatabase as pipe

# units allows us to include units in all of our calculations
from AguaClara_design.units import unit_registry as u

# utility has the significant digit display function
from AguaClara_design import utility as ut
ratio_VC_orifice= 0.62

The linear flow orifice meter was developed in 2008 by the AguaClara team at Cornell University in Ithaca, NY. It maintains a linear relationship between the height of water in the entrance tank and the overall plant flow rate, allowing an operator to easily determine how much water is flowing through the plant at any given time. This task is accomplished by mimicking the theoretical stout weir, as will be described below. 

The LFOM made possible the development of an automatic chemical dose controller. This device utilizes the linear relationship between height and flow rate provided by the LFOM, and automatically doses the proper amount of coagulant to water leaving the entrance tank. 

Below, we describe the algorithm which designs the AguaClara LFOM and then develop this algorithm using Mathcad code. 

In [50]:
# The following constants need to go into the constants file
Pi_LFOM_safety = 1.2
# pipe schedule for LFOM
SDR_LFOM = 26

In [51]:
def width_stout(HL_LFOM,z):
    return 2/((2*u.g_0*z)**(1/2)*math.pi*HL_LFOM)

In [52]:
def n_lfom_rows(FLOW,HL_LFOM):
    N_ESTIMATED = (HL_LFOM*math.pi/(2*width_stout(HL_LFOM,HL_LFOM)*FLOW)).to(u.dimensionless)
    return min(10,max(4,math.trunc(N_ESTIMATED.magnitude)))

In [53]:
FLOW = 5*u.L/u.s
HL_LFOM = 20*u.cm

In [54]:
n_lfom_rows(FLOW,HL_LFOM)

10

In [55]:
(width_stout(HL_LFOM,HL_LFOM)*FLOW).to(u.m)/u.m

<Quantity(0.008035812563075291, 'dimensionless')>

In [56]:

def dist_center_lfom_rows(FLOW,HL_LFOM):
    return HL_LFOM/n_lfom_rows(FLOW,HL_LFOM)

In [57]:
dist_center_lfom_rows(FLOW,HL_LFOM)

<Quantity(2.0, 'centimeter')>

In [58]:
# average vertical velocity of the water inside the LFOM pipe 
# at the very bottom of the bottom row of orifices
# The speed of falling water is 0.841 m/s for all linear flow orifice meters of height 20cm, independent of total plant flow rate.
def vol_lfom_pipe_critical(HL_LFOM):
    return (4/(3*math.pi)*(2*u.g_0*HL_LFOM)**(1/2)).to(u.m/u.s)

In [59]:
vol_lfom_pipe_critical(HL_LFOM)

<Quantity(0.8405802802312781, 'meter / second')>

In [60]:
def area_lfom_pipe_min(FLOW,HL_LFOM,Pi_LFOM_safety):
    return (Pi_LFOM_safety*FLOW/vol_lfom_pipe_critical(HL_LFOM)).to(u.m**2)

In [61]:
 area_lfom_pipe_min(FLOW,HL_LFOM,Pi_LFOM_safety)

<Quantity(0.007137926193497133, 'meter ** 2')>

In [62]:
def nom_diam_lfom_pipe(FLOW,HL_LFOM,Pi_LFOM_safety,SDR_LFOM):
    ID=pc.diam_circle(area_lfom_pipe_min(FLOW,HL_LFOM,Pi_LFOM_safety))
    return pipe.ND_SDR_available(ID,SDR_LFOM)

In [63]:
nom_diam_lfom_pipe(11*u.L/u.s,HL_LFOM,Pi_LFOM_safety,SDR_LFOM)

<Quantity(6.0, 'inch')>

I'm not including the functions that change the height of the LFOM as a function of flow rate because it isn't clear that those functions will be needed. It is possible that we will switch to plane LFOMs for high flowrates

In [64]:
# another possibility is to use integration to solve this problem.
# Here we use the width of the stout weir in the center of the top row
# to estimate the area of the top orifice
def area_lfom_orifices_max(FLOW,HL_LFOM):
    return FLOW*width_stout(HL_LFOM,HL_LFOM-0.5*dist_center_lfom_rows(FLOW,HL_LFOM))*dist_center_lfom_rows(FLOW,HL_LFOM)

def d_lfom_orifices_max(FLOW,HL_LFOM):
    return (pc.diam_circle(area_lfom_orifices_max(FLOW,HL_LFOM)))


In [65]:
(FLOW*width_stout(HL_LFOM,HL_LFOM-0.5*dist_center_lfom_rows(FLOW,HL_LFOM))).to(u.mm)

<Quantity(8.244569731129168, 'millimeter')>

In [66]:
from enum import Enum
class uomeasure(Enum):
    english = 0
    metric = 1

# define the constant. How do we make all of the constants available to designers?   
drill_series_uom=uomeasure

In [67]:
def drill_series(uomeasure):
    if uomeasure is uomeasure.english:
        ds=np.arange(1/32, 1/4, 1/32)
        ds=np.append(ds,np.arange(3/8, 1, 1/8))
        ds=np.append(ds,np.arange(1.25, 3.25, 1/4))
        ds=ds*u.inch
    else:
        ds=np.arange(0.5, 4.9, 0.1)
        ds=np.append(ds,np.arange(5, 19, 1))
        ds=np.append(ds,np.arange(20, 50, 2))
        ds=ds*u.mm
    return ds

In [68]:
drill_series(drill_series_uom.metric)   

<Quantity([  0.5   0.6   0.7   0.8   0.9   1.    1.1   1.2   1.3   1.4   1.5   1.6
   1.7   1.8   1.9   2.    2.1   2.2   2.3   2.4   2.5   2.6   2.7   2.8
   2.9   3.    3.1   3.2   3.3   3.4   3.5   3.6   3.7   3.8   3.9   4.
   4.1   4.2   4.3   4.4   4.5   4.6   4.7   4.8   5.    6.    7.    8.
   9.   10.   11.   12.   13.   14.   15.   16.   17.   18.   20.   22.
  24.   26.   28.   30.   32.   34.   36.   38.   40.   42.   44.   46.
  48. ], 'millimeter')>

In [69]:
# Take the values of the array, compare to x, find the index of the first value less than or equal to x
def floor_nearest(x,array):
    myindex = np.argmax(array <= x)
    return array[myindex]

# Take the values of the array, compare to x, find the index of the first value greater or equal to x
def ceil_nearest(x,array):
    myindex = np.argmax(array >= x)
    return array[myindex]

In [70]:
def lfom_drillbit_diameter(FLOW,HL_LFOM,drill_series_uom):
    return ceil_nearest(d_lfom_orifices_max(FLOW,HL_LFOM),drill_series(drill_series_uom))

In [71]:
(d_lfom_orifices_max(FLOW,HL_LFOM)).to(u.mm)

<Quantity(14.489521876859984, 'millimeter')>

In [72]:
lfom_drillbit_diameter(FLOW,HL_LFOM,drill_series_uom.english)

<Quantity(0.625, 'inch')>

In [73]:
def lfom_drillbit_area(FLOW,HL_LFOM,drill_series_uom):
    return pc.area_circle(lfom_drillbit_diameter(FLOW,HL_LFOM,drill_series_uom))

In [74]:
lfom_drillbit_area(FLOW,HL_LFOM,drill_series_uom.english)

<Quantity(0.30679615757712825, 'inch ** 2')>

In [75]:
##A bound on the number of orifices allowed in each row.  
##The distance between consecutive orifices must be enough to retain structural integrity of the pipe

def n_lfom_orifices_per_row_max(FLOW,HL_LFOM,drill_series_uom,SDR_LFOM):
    S_lfom_orifices_Min= 3*u.mm
    return math.floor(math.pi*(pipe.ID_SDR(nom_diam_lfom_pipe(FLOW,HL_LFOM,Pi_LFOM_safety,SDR_LFOM),SDR_LFOM))/(lfom_drillbit_diameter(FLOW,HL_LFOM,drill_series_uom)+S_lfom_orifices_Min))

In [76]:
n_lfom_orifices_per_row_max(FLOW,HL_LFOM,drill_series_uom.metric,SDR_LFOM)

18

In [77]:
#locations where we will try to get the target flows is in between orifices at elevation Pi.H
def flow_ramp(FLOW,HL_LFOM):
    return((np.arange((dist_center_lfom_rows(FLOW,HL_LFOM)/u.cm),(HL_LFOM/u.cm),(dist_center_lfom_rows(FLOW,HL_LFOM)/u.cm))*FLOW)*u.cm)/HL_LFOM

In [78]:
flow_ramp(FLOW,HL_LFOM)

<Quantity([ 0.5  1.   1.5  2.   2.5  3.   3.5  4.   4.5], 'liter / second')>

In [79]:
def height_lfom_orifices(FLOW,HL_LFOM,drill_series_uom):
    return np.arange(lfom_drillbit_diameter(FLOW,HL_LFOM,drill_series_uom.metric)*0.5,HL_LFOM,dist_center_lfom_rows(FLOW,HL_LFOM),dtype= object)

In [80]:
height_lfom_orifices(FLOW,HL_LFOM,drill_series_uom.metric) 

array([<Quantity(7.5, 'millimeter')>, <Quantity(27.5, 'millimeter')>,
       <Quantity(47.5, 'millimeter')>, <Quantity(67.5, 'millimeter')>,
       <Quantity(87.5, 'millimeter')>, <Quantity(107.5, 'millimeter')>,
       <Quantity(127.5, 'millimeter')>, <Quantity(147.5, 'millimeter')>,
       <Quantity(167.5, 'millimeter')>, <Quantity(187.5, 'millimeter')>], dtype=object)

In [81]:
#Calculate the flow for a given number of submerged rows of orifices
def flow_lfom_actual(FLOW,HL_LFOM,drill_series_uom,Row_Index_Submerged,N_LFOM_Orifices):
    D_LFOM_Orifices=lfom_drillbit_diameter(FLOW,HL_LFOM,drill_series_uom).to(u.m)
    FLOW_new=[]
    for i in range(Row_Index_Submerged):
        h = np.arange(dist_center_lfom_rows(FLOW,HL_LFOM),HL_LFOM,dist_center_lfom_rows(FLOW,HL_LFOM),dtype=object)
        h = h[Row_Index_Submerged].to(u.m)
        d = np.arange(0.5* D_LFOM_Orifices,HL_LFOM,dist_center_lfom_rows(FLOW,HL_LFOM),dtype=object)
        FLOW_new.append(N_LFOM_Orifices[i]*(pc.flow_orifice_vert(D_LFOM_Orifices,h-d[i],ratio_VC_orifice)))
    return sum(FLOW_new)

In [82]:
flow_lfom_actual(FLOW,HL_LFOM,drill_series_uom,5,[13,3,3,4,2,3,2,2,2])

<Quantity(3.568962891638526, 'liter / second')>

In [83]:
#Calculate number of orifices at each level given a diameter
def fric_n_lfom_orifices(FLOW,HL_LFOM,drill_series_uom,SDR_LFOM):
    FLOW_ramp_local=flow_ramp(FLOW,HL_LFOM)
    D_LFOM_Orifices=lfom_drillbit_diameter(FLOW,HL_LFOM,drill_series_uom)
    H_ramp_local=np.arange(D_LFOM_Orifices*0.5,HL_LFOM,dist_center_lfom_rows(FLOW,HL_LFOM),dtype=object)
    n=[]
    for i in range (len(H_ramp_local)-1):
        h=np.arange(dist_center_lfom_rows(FLOW,HL_LFOM),HL_LFOM,dist_center_lfom_rows(FLOW,HL_LFOM),dtype=object)
        d=H_ramp_local
        if nom_diam_lfom_pipe(FLOW,HL_LFOM,Pi_LFOM_safety,SDR_LFOM)<=12*u.inch:
            n.append(min(max(0,round((FLOW_ramp_local[i]-flow_lfom_actual(FLOW,HL_LFOM,drill_series_uom,i,n))/pc.flow_orifice_vert(D_LFOM_Orifices,h[i]-d[i],ratio_VC_orifice))),n_lfom_orifices_per_row_max(FLOW,HL_LFOM,drill_series_uom,SDR_LFOM)))
        else:
             n.append(max(0,round((FLOW_ramp_local[i]-flow_lfom_actual(FLOW,HL_LFOM,drill_series_uom,i,n))/pc.flow_orifice_vert(D_LFOM_Orifices,h[i]-d[i],ratio_VC_orifice))))
    return n
                     

In [84]:
print(fric_n_lfom_orifices(FLOW,HL_LFOM,drill_series_uom,SDR_LFOM))

[<Quantity(9.0, 'dimensionless')>, <Quantity(4.0, 'dimensionless')>, <Quantity(3.0, 'dimensionless')>, <Quantity(2.0, 'dimensionless')>, <Quantity(3.0, 'dimensionless')>, <Quantity(1.0, 'dimensionless')>, <Quantity(3.0, 'dimensionless')>, <Quantity(1.0, 'dimensionless')>, <Quantity(1.0, 'dimensionless')>]


In [85]:
#This function calculates the error of the design based on the differences between the predicted flow rate
#and the actual flow rate through the LFOM.
def flow_lfom_error(FLOW,HL_LFOM,drill_series_uom,SDR_LFOM):
    N_lfom_orifices=fric_n_lfom_orifices(FLOW,HL_LFOM,drill_series_uom,SDR_LFOM)
    FLOW_lfom_error=[]
    for j in range (len(N_lfom_orifices)-1):
        FLOW_lfom_error.append((flow_lfom_actual(FLOW,HL_LFOM,drill_series_uom,j,N_lfom_orifices)-flow_ramp(FLOW,HL_LFOM)[j])/FLOW)
    return FLOW_lfom_error

In [86]:
print(flow_lfom_error(FLOW,HL_LFOM,drill_series_uom,SDR_LFOM))

[<Quantity(-0.1, 'dimensionless')>, <Quantity(-0.04281022675343118, 'dimensionless')>, <Quantity(-0.03014536546205875, 'dimensionless')>, <Quantity(-0.023627100599359574, 'dimensionless')>, <Quantity(-0.028341082805428908, 'dimensionless')>, <Quantity(-0.013856233840633347, 'dimensionless')>, <Quantity(-0.027031604731054416, 'dimensionless')>, <Quantity(-0.008034482759309292, 'dimensionless')>]


In [87]:
#This funciton returns the maximum error, the absolute value of the errors is take into account positive 
#and negative errors
x= max(flow_lfom_error(FLOW,HL_LFOM,drill_series_uom,SDR_LFOM))
y=x**2
FLOW_LFOM_ERROR_MAX=y**1/2

In [88]:
print (FLOW_LFOM_ERROR_MAX)

3.227645660481912e-05 dimensionless


In [89]:
def flow_lfom_ideal(FLOW,HL_LFOM,H):
    flow_lfom_ideal=(FLOW*H)/HL_LFOM
    return flow_lfom_ideal

In [90]:
print (flow_lfom_ideal(FLOW,HL_LFOM,3))

0.75 liter / centimeter / second


In [91]:
def flow_lfom(FLOW,HL_LFOM,drill_series_uom,SDR_LFOM,H):
    D_lfom_orifices=lfom_drillbit_diameter(FLOW,HL_LFOM,drill_series_uom)
    H_submerged=np.arange(H-0.5*D_lfom_orifices,HL_LFOM,H-dist_center_lfom_rows(FLOW,HL_LFOM),dtype=object)
    N_lfom_orifices=fric_n_lfom_orifices(FLOW,HL_LFOM,drill_series_uom,SDR_LFOM)
    flow=[]
    for i in range (len(H_submerged)):
        flow.append(pc.flow_orifice_vert(D_lfom_orifices,H_submerged[i],ratio_VC_orifice)*N_lfom_orifices[i])
    return sum (flow)

In [92]:
print(flow_lfom(FLOW,HL_LFOM,drill_series_uom,SDR_LFOM,5*u.cm))

3.04063042951175 liter / second


In [93]:
HEIGHT_LFOM_ORIFICES=height_lfom_orifices(FLOW,HL_LFOM,drill_series_uom)

In [94]:
if FLOW==1.6*(u.L/u.s):
   NOM_DIAM_RAPID_MIX_pipe=2*u.inch
else:
   NOM_DIAM_RAPID_MIX_pipe=nom_diam_lfom_pipe(11*u.L/u.s,HL_LFOM,Pi_LFOM_safety,SDR_LFOM)

In [95]:
N_LFOM_ORIFICES=fric_n_lfom_orifices(FLOW,HL_LFOM,drill_series_uom,SDR_LFOM)

In [96]:
N_LFOM_ROWS=len(N_LFOM_ORIFICES)